In [ ]:
# Setting up environment
# ----------------------
# Import libraries
import psycopg2
import cloudscraper
import time
from bs4 import BeautifulSoup
import sys
import os
import hashlib
import string
import re

# Set custom path for secrets
cwd = os.getcwd()
secrets = os.path.join(cwd,"../Secrets")
print(secrets)
sys.path.insert(0,secrets)

<h4 style="color:orange"> Make database of links to job postings. </hi>


In [ ]:
# Logging
# -------
log_dir = os.path.join(os.getcwd(),'Logs')
print(log_dir)
file_num = len([file for file in os.listdir(log_dir) if 'log' in file])
log_file = 'log' + str(file_num) + '.txt'
log_path = os.path.join(log_dir,log_file)
print(log_path)
# Function to write to log files
def w2f(text):
    if type(text) != str:
        text = str(text)
    with open(log_path,'a+') as f:
        f.write(text)
        f.write("\n")

In [ ]:
# Feedback for sql commands
# -------------------------
# Function to run try-except wrapped sql commands 
def run_sql (conn, sql_cmnd):
    cur = conn.cursor()
    try:
        cur.execute(sql_cmnd)
        print('success: ' + sql_cmnd[:75])
        w2f('success: ' + sql_cmnd[:100])
    except:
        print('FAIL: ' + sql_cmnd[:75])
        w2f('FAIL: ' + sql_cmnd[:100])
    conn.commit()
    return cur
    

In [ ]:
# Connecting to local database
# ----------------------------
# Connect to database
import crawler_pg_secrets as p
pg_params = {
    'host' : p.host,
    'user' : p.user,
    'dbname' : p.dbname,
    'password' : p.password,
    'port' : p.port
}
conn =  psycopg2.connect(**pg_params)
cur = conn.cursor()
conn.commit()

In [ ]:
# Creating job url table
# ----------------------
table_name = 'indeed_horizon'

# Drop previous horizon table
sql_cmnd = "drop table " + table_name + ";"
run_sql(conn,sql_cmnd)

# Create new horizon table
sql_cmnd = "create table "\
    + table_name\
    + " (id serial, url varchar(2046), visited integer);"
run_sql(conn,sql_cmnd)

# Create MD5 hashed index
sql_cmnd = 'CREATE UNIQUE INDEX url_md5 ON indeed_horizon(MD5(url));'
run_sql(conn,sql_cmnd)

In [ ]:
# Generate page urls
# -------------------
# Defining indeed endpoint, example: jobs?q=data+analyst&l=Toronto'&start=40
def create_end_url(job = 'data analyst', location = 'toronto', page = 3):
    connector_01 = '&l='
    connector_02 = "&start="
    job_url = job.strip().replace(" ","+")
    page_url = (page - 1) *10
    location_url = location
    end_url = '/jobs?q=' + job_url + connector_01 + location_url + connector_02 + str(page_url)
    print(end_url)
    w2f(("Initial end url: " + end_url))
    return end_url

# Defining indeed full url, example: https://ca.indeed.com/jobs?q=data+analyst&l=Toronto'&start=40
def create_full_url(end_url):
    base_url = 'https://ca.indeed.com'
    full_url =  base_url + end_url
    print(full_url)
    w2f(('Initial full url: ' + full_url))
    return full_url

end_url = create_end_url('data analyst', 'toronto',1) # testing url = create_url('data entry', 'ottawa',9)
full_url = create_full_url(end_url)


In [ ]:
#Build table of horizons
# -----------------------
# Function to enter url into database
def db_input (url, visited = 0, table_name = 'indeed_horizon'):
    sql_cmnd = "insert into indeed_horizon (url, visited) values ('" + url + "'," +str(visited)+ ");"
    exit_status = run_sql(conn,sql_cmnd)
    return exit_status

# Extract urls from indeed
# ------------------------
# Parse page (from response object) for job urls
def scrape_soup(soup,advert_count):
    print("advert type count begining: ", type(advert_count))
    job_url_base = 'https://ca.indeed.com'
    ident='jcs-JobTitle css-jspxzf eu4oa1w0'
    class_link = soup.find_all(class_=ident) # Div class containing job link
    for link in class_link:
        advert_count += 1
        job_url_end = link.get('href')
        job_url = job_url_base + job_url_end
        exit_status = db_input(job_url)
        #print("advert_count before, scrape_soup: ",  advert_count)
        # if exit_status == 0:
        #     advert_count += 1
        #print("advert_count after, scrape_soup: ",  advert_count)
        print(advert_count)
        w2f(("Job advert number: " + str(advert_count)))
        w2f(("Full job advert url" + job_url))
    return advert_count

In [ ]:
# Scraping with cloudscraper
# --------------------------
# Scraping instance
print(full_url)
scraper = cloudscraper.create_scraper()

# Cycling through pages
exit_flag = 0
advert_count = 0
page = 0
max_pages = 10
while (exit_flag == 0 and page < max_pages):
    page += 1
    print("page:",page)
    response = scraper.get(full_url)
    soup = BeautifulSoup(response.text,"html.parser")
    # Extract adverts from page
    print("advert_count before, in page loop: ",  type(advert_count))
    advert_count = scrape_soup(soup, advert_count)
    print("advert_count after, in page loop: ",  type(advert_count))
    print(response.status_code)
    # Find next page
    try:
        next_page_resultset = soup.find_all(attrs = {'data-testid':'pagination-page-next'})
        print(next_page_resultset)
        next_page_end_link = next_page_resultset[0]['href']
        print("next page link:",next_page_end_link)
        w2f(next_page_end_link)
        time.sleep(1)
        full_url = create_full_url(next_page_end_link)
    except:
        exit_flag = 1
        # print("T5")


<h4 style="color:orange"> Make local cache of job postings. </hi>

In [ ]:
# rm
# --
scraper = cloudscraper.create_scraper()
# Get job posting addresses from database.
# ----------------------------------------
def save_job_adv (link):
    cldflr_bottrap_msg = 'Just a moment...'
    fpath = os.path.join(os.getcwd(), 'Cache', hashlib.md5((link).encode()).hexdigest())
    get_page = True
    attempt = 0
    print(fpath)
    while get_page and attempt <=3:
        attempt += 1
        response = scraper.get(link)
        soup = BeautifulSoup(response.text,"html.parser")
        # Check if request is arrested by cloudflare bot check
        pg_title = soup.title.text.replace('\n','').strip()
        if not(pg_title == cldflr_bottrap_msg):
            get_page = False
            with open(fpath, 'w') as f:
                f.write(soup.prettify())
        else:
            print("Sleeping")
            time.sleep(10)
        print(type(soup.prettify()))
        


sql_cmnd="select * from indeed_horizon;"
cursor = run_sql(conn,sql_cmnd)
count = 0
for row in cursor.fetchall():
    if count < 50:
        try: 
            count += 1
            link = row[1]
            print('-'*20)
            print(hashlib.md5((link).encode()).hexdigest())
            print(link)
            save_job_adv(link)
            time.sleep(1)
        except:
            print("Page retreival failed.")
    else:
        break



<h4 style="color:orange"> Analyzing cached data. </hi>

In [ ]:
# Create table to stor job text
sql_cmnd = "create job_text (\
    id serial,\
    word varchar(512),\
    md5 foreign key )"
run_sql(conn,sql_cmnd)


In [ ]:
pages = os.listdir('Cache')
skills = ['sql', 'excel','tableau', 'powerbi', 'python', 'r', 'sas', 'word', 'powerpoint', 'ms', 'microsoft']
skills_2 = ['sql', 'excel','tableau', 'powerbi', 'python', 'r', 'sas', 'word', 'powerpoint', 'ms', 'microsoft', 'data analyst']
pattern = '[' + string.punctuation + ' ' + ']' + '+'
word_dict = {}
skill_dict ={}
skill_dict_2 = {}

# Implementation option 1 - ideal for keywords consisting of only 1 word eg. "python"
# Excellent for keywords comprising of a single letter eg. 'r'
# Not good for keywords made up of more than one word eg. "microsoft office"
# ----------------------------------------------------------------------------------
# Regex pattern for splitting based on punctuation and spaces
# pattern = [!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ]+

# print(pages)
for page in pages[:]:
    # page = '63e00cff7278e8f28a834b8c2c16f9e8'
    # print(page)
    with open('Cache/'+page,'r') as f:
        soup = BeautifulSoup(f)
        # print(soup.prettify())
    words = [word for word in re.split(pattern,soup.text.lower().replace('\n',' ')) if len(word) > 1]
    # print('-*'*50)
    with open ('Test/token-text.txt', 'a+') as f:
        for word in words:
            f.write(str(word))
            f.write("\n")
    # print('-*'*50)
    for word in words:
        word_dict[word] = word_dict.get(word, 0) + 1
print(len(word_dict))

for skill in skills:
    print(skill, ":", word_dict.get(skill, 0), "\t\t", "Adv: ", (word_dict.get(skill,0)/len(pages))*100)
print("Total number of job advertisments analysed: ", len(pages))
# print((word_dict['sql']))
# print(word_dict)

# Implementation option 1 - ideal for keywords consisting of multiple words eg. "ms office"
# Terrible for keywords comprising of a single letter eg. 'r'
# Okayish for keywords made up of only one word eg. "python"
# ----------------------------------------------------------------------------------
for page in pages[:]:
   with open('Cache/'+page,'r') as f:
        soup = BeautifulSoup(f)
        page_text = soup.text.lower()#.replace('\n',' ')
        # print('-*'*50)
        with open ('Test/full-text.txt', 'a+') as f:
            f.write(page_text)
        # print('-*'*50)
        for skill in skills_2:
            # print(page_text.count(skill))
            skill_dict_2[skill] = skill_dict_2.get(skill,0) + page_text.count(skill)
            # if skill in page_text:
            #     skill_dict_2[skill] = skill_dict_2.get(skill,0) + 1
print(skill_dict_2)



<h4 style="color:hotpink"> Testing. </h4>

In [ ]:
result = hashlib.md5(b'meow')
print (hashlib.md5(b"whatever your string is").hexdigest())
print (type((hashlib.md5(b"whatever your string is").hexdigest())))
var1='woof'
var1.encode()

punctuation = []
for item in string.punctuation:
    punctuation.append(item)
print(punctuation)

var2= "This. is a, test-of (things"
pattern='[-,*)(). ]+'
re.split(pattern, var2)
string.punctuation

var2='this.is/a"te$stsr'
'is/a' in var2
var2.count("ishj")

In [ ]:
count = 0
repeat = True
while repeat:
    time.sleep(5)
    print(count)
    count += 1
    if count > 3:
        repeat = False

In [ ]:
a=not(True)
print(a)